Bueno, paso a paso. Primero hemos tenido que conocer el dispositivo del pc que graba audio. He dejado una emidión de rtve1 en internet que suene en el navegador. Y después he tenido que conocer qué dispositivo reconoce/graba audio. Para ver estos dispoitvos disponibles, he ejecutado en el bash (powerhecll) el siguiente comando: 

ffmpeg -list_devices true -f dshow -i dummy

He visto esta salida:

[dshow @ 000002338a71db00] "Microphone (Realtek(R) Audio)" (audio)

Habiendo tenido esta salida, vamos a correr el siguiente comando para grabar 10 segundos de audio como test. Recordamos que tenemos una web de rtve viendo la programacin en directo:

ffmpeg -f dshow -i "audio=Microphone (Realtek(R) Audio)" -t 10 test.wav

Efectivamente, en el directorio del bash donde he estado ejecutrando esos comandos, se ha guardado un archico .wav con la grabación. 

Ahora, pues, vayamos a correr el script que debe ir guardando el audio con un timestamp:



In [4]:
import os
import datetime
import subprocess
import time 

def grabar_audio_con_timestamps(duracion_segundos, archivo_salida="emision_audio.wav", intervalos=10):
    """Graba audio y guarda timestamps en un archivo separado."""
    timestamp_inicial = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    archivo_audio = f"{timestamp_inicial}_{archivo_salida}"
    archivo_timestamps = f"{timestamp_inicial}_timestamps.txt"

    print(f"Grabando audio con timestamps... Duración: {duracion_segundos} segundos")

    # Iniciar FFmpeg en un proceso separado
    comando = [
        "ffmpeg",
        "-f", "dshow",
        "-i", "audio=Stereo Mix (Realtek High Definition Audio)",
        "-t", str(duracion_segundos),
        archivo_audio
    ]
    proceso = subprocess.Popen(comando, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Generar y guardar timestamps mientras FFmpeg graba
    with open(archivo_timestamps, "w") as f:
        inicio = datetime.datetime.now()
        while (datetime.datetime.now() - inicio).seconds < duracion_segundos:
            timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            f.write(f"{timestamp}\n")
            print(f"Timestamp registrado: {timestamp}")
            time.sleep(intervalos)

    proceso.wait()  # Esperar a que FFmpeg termine
    print(f"Grabación y timestamps completados. Archivos: {archivo_audio}, {archivo_timestamps}")
    return archivo_audio, archivo_timestamps


In [5]:
grabar_audio_con_timestamps(60, "web_emision.wav", 5)


Grabando audio con timestamps... Duración: 60 segundos
Timestamp registrado: 2024-12-17 23:54:28
Timestamp registrado: 2024-12-17 23:54:33
Timestamp registrado: 2024-12-17 23:54:38
Timestamp registrado: 2024-12-17 23:54:43
Timestamp registrado: 2024-12-17 23:54:48
Timestamp registrado: 2024-12-17 23:54:53
Timestamp registrado: 2024-12-17 23:54:58
Timestamp registrado: 2024-12-17 23:55:03
Timestamp registrado: 2024-12-17 23:55:08
Timestamp registrado: 2024-12-17 23:55:13
Timestamp registrado: 2024-12-17 23:55:18
Timestamp registrado: 2024-12-17 23:55:23
Grabación y timestamps completados. Archivos: 20241217_235428_web_emision.wav, 20241217_235428_timestamps.txt


('20241217_235428_web_emision.wav', '20241217_235428_timestamps.txt')

Se ha guardado un audio pero es lo capturado por le micrófono, lo cual no ha captado nada de la emision de tve que tenía en la web. Además ha guardado un -txt de los timestamps cada 5 segundos, pero no se puede relacionar con la grabación. Vamos apreguntarlo como hacerlo, que posiblmeente será guardar archivos de audio con el timestamp impregnado en su nombre:

In [6]:
import subprocess
import time
from datetime import datetime

def grabar_audio_segmentado(duracion_segmento, duracion_total, carpeta_salida="grabaciones"):
    """
    Graba audio en segmentos de tiempo definidos, nombrando los archivos con la hora local.
    
    Args:
        duracion_segmento (int): Duración de cada segmento en segundos.
        duracion_total (int): Duración total de la grabación en segundos.
        carpeta_salida (str): Carpeta donde se guardarán los archivos de audio.
    """
    import os

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    tiempo_inicial = time.time()
    while time.time() - tiempo_inicial < duracion_total:
        # Obtener timestamp actual para el nombre del archivo
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_archivo = os.path.join(carpeta_salida, f"audio_{timestamp}.wav")

        # Comando FFmpeg para grabar el audio
        comando = [
            "ffmpeg",
            "-f", "dshow",                   # Modo de entrada directa
            "-i", "audio=\"Microphone (Realtek(R) Audio)\"",  # Dispositivo de entrada
            "-t", str(duracion_segmento),    # Duración del segmento
            "-y",                            # Sobrescribir archivos existentes
            nombre_archivo
        ]

        print(f"Grabando: {nombre_archivo}")
        proceso = subprocess.run(comando, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        if proceso.returncode != 0:
            print(f"Error al grabar el archivo {nombre_archivo}. Verifica el dispositivo de entrada.")
            break

        print(f"Grabación completada: {nombre_archivo}")

    print("Grabación total completada.")




In [7]:
grabar_audio_segmentado(duracion_segmento=10, duracion_total=60)


Grabando: grabaciones\audio_20241217_235528.wav
Error al grabar el archivo grabaciones\audio_20241217_235528.wav. Verifica el dispositivo de entrada.
Grabación total completada.


Vale, bueno, he conseguido que una emisión que sunee en mi pc se grabe en un audio perfectamente. Esto es, configurando la grabacion de sonido en Stereo Mix a través de windows. El video en el que he podido aprender esto es el siguiente: https://www.youtube.com/watch?v=xhuwo-q0fEY

EN la primera parte del video explica como hacerlo. Al realizarlo y al ejecutar el comando anterior en la powershell se ha visualizado el dispositvo configurado:

ffmpeg -list_devices true -f dshow -i dummy

con este comnado hemos comprobado que existía el Stereo Mix después de haber seguido los passos del video. Por lo que hemos ejecutado de nuevo el siguiente comando:

ffmpeg -f dshow -i "audio=Stereo Mix (Realtek(R) Audio)" -t 10 test.wav

Y hemos recibido una graabación impecable de lo que queríamos: capturar el audio de un video ejecutado en el navegador.

Ahora, vamos a ejecutar el script de python a ver cómo funciona:

Vemos que el 'dispositivo' Stereo Mix lo hemos insertado en la siguiente líena del código:

 "-i", "audio=Stereo Mix (Realtek(R) Audio)",  # Cambia por tu dispositivo

 Corramos el script de python:





In [8]:
import os
import subprocess
import time
from datetime import datetime  # Asegúrate de importar correctamente

def grabar_audio_segmentado(duracion_segmento=10, duracion_total=60, carpeta_salida="grabaciones"):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)
    
    tiempo_inicial = time.time()
    while time.time() - tiempo_inicial < duracion_total:
        # Obtener timestamp actual para el nombre del archivo
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_archivo = os.path.join(carpeta_salida, f"audio_{timestamp}.wav")
        
        # Comando FFmpeg para grabar el audio
        comando = [
            "ffmpeg",
            "-f", "dshow",
            "-i", "audio=Stereo Mix (Realtek(R) Audio)",  # Cambia por tu dispositivo
            "-t", str(duracion_segmento),
            "-y", nombre_archivo
        ]
        
        # Ejecutar el comando FFmpeg
        print(f"Grabando segmento: {nombre_archivo}")
        subprocess.run(comando, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    print("Grabación completada.")





In [9]:
grabar_audio_segmentado(duracion_segmento=10, duracion_total=60)

Grabando segmento: grabaciones\audio_20241217_235528.wav
Grabando segmento: grabaciones\audio_20241217_235540.wav
Grabando segmento: grabaciones\audio_20241217_235552.wav
Grabando segmento: grabaciones\audio_20241217_235605.wav
Grabando segmento: grabaciones\audio_20241217_235617.wav
Grabación completada.


Muy bien, he conseguido grabar audio que se ejecutaba en la web (aunque puede ser cualeuir audio que se ejecute en el epc) y guardarlo en una carpeta que luego querremos transcribir. La prueba de conceto número 2 (dos) puede considerrse terminada. 

Ya podemos grabar cualquier audio en el pc y guardarlo como archivo fraccinado de 10 segundos. La siguietne prueba es a la vez que guardamos en archivo, transcirbirlo segun se vaya guardando. Una vez transcrito, guardarlo también. 

Y la siguietne prueba de conepto, debería ser, cómo eficientamos el guardado de las transcripciones. Pero para no adelantarnos, siguietne objetivo: grabar audio > transcribir > guardar transcripción. La transcripción la guardaremos en un formato de texto en la siguiente PoC, sin embargo, el objetivo más adelante es guardarle en un objeto más adecuado e incluso, en una BBDD.

In [10]:
grabar_audio_segmentado(duracion_segmento=10, duracion_total=60)

Grabando segmento: grabaciones\audio_20241217_235629.wav
Grabando segmento: grabaciones\audio_20241217_235641.wav
Grabando segmento: grabaciones\audio_20241217_235653.wav
Grabando segmento: grabaciones\audio_20241217_235705.wav
Grabando segmento: grabaciones\audio_20241217_235718.wav
Grabación completada.
